# Telegram deep fake bot

[Github repo](https://github.com/piguet/deeptelegram/edit/master/DeepTelegram.ipynb)

**Clone deep-fake repository**

In [0]:
!git clone https://github.com/AliaksandrSiarohin/motion-cosegmentation motion-co-seg

In [0]:
cd motion-co-seg/

**Mount your Google drive folder on Colab**

In [0]:

from google.colab import drive
drive.mount('/content/gdrive')

**Add folder https://drive.google.com/drive/folders/1SsBifjoM_qO0iFzb8wLlsz_4qW2j8dZe  to your google drive.**

**Load the deep fake libraries**

In [0]:
import imageio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from skimage.transform import resize
from skimage.util import crop
from IPython.display import HTML
import warnings
warnings.filterwarnings("ignore")

from part_swap import load_checkpoints
from part_swap import make_video
from skimage import img_as_ubyte
cpu = False


In [0]:
from part_swap import load_checkpoints

reconstruction_module, segmentation_module = load_checkpoints(config='config/vox-256-sem-5segments.yaml', 
                                               checkpoint='/content/gdrive/My Drive/motion-supervised-co-segmentation/vox-5segments.pth.tar',
                                               blend_scale=1)

**Install the telegram bot framework**

In [0]:
!pip install pyTelegramBotAPI

## Make it a telegram bot

contact the [@BotFather](https://t.me/BotFather) on telegram and create a new bot with the command `/newbot` 

After you've given the bot a name and username you should receive a message that looks like this ![alt text](https://i.imgur.com/BT8hHEL.png)


Now copy and the paste the token below in the `API_TOKEN`.

Now you can run this cell and start chatting with the bot.

In [0]:
import telebot
import requests
import shutil
import subprocess
import os.path

API_TOKEN = ''
bot = telebot.TeleBot(API_TOKEN)
settings = {}
enabled = []

def isEnabled(id):
  return id in enabled if len(enabled)>0 else True

@bot.message_handler(commands=['start', 'help'])
def send_welcome(message):
  if isEnabled(message.chat.id):
	  bot.reply_to(message, "Hi send me a photo (with square aspect ratio), you will then be able to use this face\nSwitch output between video mode and video note mode with /output\nSwitch relative position with /relative")

def get(key,id_utente,default_value=False):
  if not id_utente in settings:
    settings[id_utente]={}
  if not key in settings[id_utente]:
    settings[id_utente][key]=default_value
  return settings[id_utente][key]

def set(key,value,id_utente):
  get(key,id_utente) #init
  settings[id_utente][key]=value


@bot.message_handler(commands=['output'])
def videomode(message):
  if isEnabled(message.chat.id):
    if get('mode',message.chat.id):
      set('mode',False,message.chat.id)
    else:
      set('mode',True,message.chat.id)
    bot.reply_to(message, f"Normal video mode is {get('mode',message.chat.id)}.")

@bot.message_handler(commands=['hair'])
def videomode(message):
  if isEnabled(message.chat.id):
    if get('hair',message.chat.id):
      set('hair',False,message.chat.id)
    else:
      set('hair',True,message.chat.id)
    bot.reply_to(message, f"Hair video mode is {get('hair',message.chat.id)}.")

@bot.message_handler(commands=['relative'])
def relativemode(message):
  if isEnabled(message.chat.id):
    if get('relative',message.chat.id,default_value=True):
      set('relative',True,message.chat.id)
    else:
      set('relative',False,message.chat.id)
    bot.reply_to(message, f"I've changed the relative mode to {get('relative',message.chat.id)} correctly.")

@bot.message_handler(content_types=['photo'])
def download_pic(message):
  if isEnabled(message.chat.id):
    file_info = bot.get_file(message.photo[0].file_id)
    file = requests.get('https://api.telegram.org/file/bot{0}/{1}'.format(API_TOKEN, file_info.file_path))
    id = message.chat.id;
    open(f'../src{id}.jpg', 'wb').write(file.content)
    bot.reply_to(message, "Perfect, now send me a video note or a video! (for best results keep head movements to a minimum and keep a static background)")

@bot.message_handler(content_types=['video_note','video'])
def download_video(message):
  if isEnabled(message.chat.id):
    try:
      id = message.chat.id;
      file_info = bot.get_file(message.video_note.file_id if message.content_type == 'video_note' else message.video.file_id)
      print('https://api.telegram.org/file/bot{0}/{1}'.format(API_TOKEN, file_info.file_path));
      file = requests.get('https://api.telegram.org/file/bot{0}/{1}'.format(API_TOKEN, file_info.file_path))
      open(f'../target{id}.mp4', 'wb').write(file.content)
      #extract audio
      subprocess.call(['ffmpeg', '-i', f'../target{id}.mp4', '-vn', '-acodec','copy', f'../out{id}.aac'])
      if os.path.exists(f'../src{id}.jpg'):
        bot.reply_to(message, "I'm generating the deep fake...")
        bot.send_chat_action(id, 'record_video')
        source_image = imageio.imread(f'../src{id}.jpg')
        source_image = resize(source_image, (256, 256))[..., :3]
      else:
        bot.reply_to(message, "You have to send me an image first!")
        return
      try:
        driving_video = imageio.mimread(f'../target{id}.mp4')
      except Exception as e:
        reader = imageio.get_reader(f'../target{id}.mp4')
        driving_video = []
        try:
            for im in reader:
                driving_video.append(im)
        except RuntimeError:
            pass
      
      driving_video = [resize(frame, (256, 256))[..., :3] for frame in driving_video]


      #predictions = make_animation(source_image, driving_video, generator, kp_detector, relative=get('relative',message.chat.id,default_value=True),cpu=cpu)
      if get('hair',message.chat.id):
        predictions = make_video(swap_index=[1,2,5], source_image = source_image, target_video = driving_video,
                             segmentation_module=segmentation_module, reconstruction_module=reconstruction_module)
      else:
        predictions = make_video(swap_index=[1,2], source_image = source_image, target_video = driving_video,
                             segmentation_module=segmentation_module, reconstruction_module=reconstruction_module)
      
      
      imageio.mimsave(f'../generated{id}.mp4', [img_as_ubyte(frame) for frame in predictions])
      #normal speed
      subprocess.call(['ffmpeg', '-itsscale','0.33334', '-i', f'../generated{id}.mp4', '-c','copy', f'../generated_fast{id}.mp4'])
      #add audio
      subprocess.call(['ffmpeg', '-i', f'../generated_fast{id}.mp4', '-i',f'../out{id}.aac', '-c', 'copy','-map','0:v:0','-map','1:a:0', f'../tosend{id}.mp4'])
      videonote = open(f'../tosend{id}.mp4', 'rb')
      if get('mode',message.chat.id):
        bot.send_video_note(id, videonote)
      else:
        bot.send_video(id,videonote)
    finally:
      #cleanup
      if os.path.exists(f'../target{id}.mp4'):
        os.remove(f'../target{id}.mp4')
      if os.path.exists(f'../generated{id}.mp4'):
        os.remove(f'../generated{id}.mp4')
      if os.path.exists(f'../generated_fast{id}.mp4'):
        os.remove(f'../generated_fast{id}.mp4')
      if os.path.exists(f'../tosend{id}.mp4'):
        os.remove(f'../tosend{id}.mp4')
      if os.path.exists(f'../out{id}.aac'):
        os.remove(f'../out{id}.aac')
      if os.path.exists(f'../src{id}.jpg'):
        os.remove(f'../src{id}.jpg')

bot.polling()